# Deep Reinforcement Learning - Continuous Control





# Overview

The goal of this project is to train an agent to achieve an average reward of atleast +30 over 100 consecutive episodes in the provided environment.

The environment provided is that of a Unity ML Agents scenario.

In this environment, a double-jointed arm can move to target locations. A reward of +0.1 is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of 33 variables corresponding to position, rotation, velocity, and angular velocities of the arm. Each action is a vector with four numbers, corresponding to torque applicable to two joints. Every entry in the action vector should be a number between -1 and 1.

Two versions of the environment are provided here. One where there is only one agent being trained at a time and another where 20 agents are simultaneously being trained

# Implementation

The project was solved using Deep Reinforcement Learning using a DDPG(Deep Deterministic Policy Gradient) Algorithm.

The base code of the project is derived from the solution provided in Udacity Deep Learning Nanodegree Github repo for solving the pendulum scenario from OpenAI Gym.

https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum

The same solution was modified and updated for the Unity ML Agents environment provided.

* The notebooks **Continuous_Control_SingleAgent.ipynb** and **Continuous_Control_MultiAgent.ipynb** contains the implementation for training and visualising the untrained agent initially for the Single and 20 Agents respectively. Then the training code is implemented.

* **ddpg_agent.py** contains the code to understand and determine how the agent interacts with the environment and learns to optimize the reward.

* **model.py** contains the architecture of the deep learning model used in this implementation.


# Learning Algorithm

The Deep Deterministic Policy Gradient algorithm(DDPG) was chosen to solve the agent-environment interaction.

In general, the more commonly known **Q-Learning** method utilizes a state-action return value and from there chooses the actions which maximise the expected reward, hence reaching the optimal policy.

**Policy gradients** on the other hand tries to direcly map the states to the actions where a neural network can be used as a function approximator, by increasing the probabilities of the actions which yield higher returns, leading to the optimal policy.

DDPG combines these two ideas in the form of an actor-critic algorithm. It is based on the deterministic policy gradient that can operate over continuous action spaces.

The Actor network here maps the states to the actions while the Critic network tries to estimate the Q function based on the actions selected by the Actor. The actor on the other hand learns the policy based off the Critic network.

It also utilizes the replay buffer to learn from past experiences in the way DQN Algorithm does.
Furthermore Ornstein-Uhlenbeck process is utilized to generate a noise in order to encourage exploration.

Another implementation borrowed from the DQN Algorithm is the usage of the target and local networks which here has been used for both the actor and critic network.

### Model Architecture
There are two model architectures used here defined in the file model.py :

#### Actor

* The model has 2 fully connected layers with 400 and 300 nodes respectively.
* It takes in an input equal to the state size provided which in this case is 33
* Relu activation function is used betweeen the two layers
* Batch normalisation has been applied since it has been proven that it speeds up the learning in case of DDPG
* The output node of this layer is of size equal to the action size


#### Critic

* The model has 2 fully connected layers with 400 nodes in the first layer and (300+action_size) in the second.
* It takes in an input equal to the state size provided which in this case is 33
* Relu activation function is used betweeen the two layers
* Batch normalisation has been applied since it has been proven that it speeds up the learning in case of DDPG
* The output node of this layer is of size equal to 1 which means that it outputs the expected return value
* torch.cat operation is applied between the first two layers to denote the mapping between the states and actions as defined in the algorithm

### Agent

The agent is defined in the agent.py file. It is the Deep Q-Learning agent which interacts with the environment. It references the local and target network from the model defined in model.py.

It contains four methods:

**Step**: Here the agent saves the experiences in the replay memory. After a certain set of predefined intervals, it also causes the network to learn from the replay buffer a certain number of times.

**Act**: Here the agent returns the action determined by the local Actor Network. The output is of size 4 corresponding to each actions but in the range of -1 to 1 as expected by the network. Furthermore, noise is added via the Ornstein-Uhlenbeck process to encourage exploration.

**Learn**: This is where the agent actually learns. 

#### Critic network learning
We randomly sample a batch from the experience buffer in the form of (states, actions, rewards, next_states, dones) and pass on the next_states to the actor target network to determine the next set of actions which in return is passed on to the critic target network.

The return from the critic target network is actually the Q values determined by the target network.

Again, like in the DQN Algorithm, we compute the expected Q values from these next states and compute the Mean Squared Error loss between Q_targets and Q_expected and update the critic target network accordingly.

# Actor network learning
Here we get the predicted actions from the local actor network based on the current states.
The loss is computed as the mean of the Q values corresponding to the different state action pairs.
We use the negative sign here because we want to maximise the gradients and hence using gradient ascent.

Then we run the soft_update function to update the target network with the local network parameters

**Soft_update**: Here we update the target networks with the local networks parameters using the formula
                    
                    θ_target = τ*θ_local + (1 - τ)*θ_target

### Hyperparameters

The hyperparameters used to train the agent are:



* BUFFER_SIZE = int(1e6)  # replay buffer size
* BATCH_SIZE = 128        # minibatch size
* GAMMA = 0.99            # discount factor
* TAU = 1e-3              # for soft update of target parameters
* LR_ACTOR = 2e-4         # learning rate of the actor 
* LR_CRITIC = 2e-4        # learning rate of the critic
* WEIGHT_DECAY = 0        # L2 weight decay
* UPDATE_EVERY=20         # Update interval
* NUM_UPDATES=10          # Number of learning steps after every predefined interval in UPDATE_EVERY

# Plot of Rewards

The plot of average rewards for every 100 episodes for **Single Agent** are listed below.

* Episode 100	Average Score: 5.44	Score: 11.59
* Episode 200	Average Score: 20.72	Score: 18.65
* Episode 300	Average Score: 26.33	Score: 32.50
* Episode 400	Average Score: 27.69	Score: 31.50
* Episode 490	Average Score: 30.10	Score: 35.93
* Environment solved in 490 episodes!	Average Score: 30.10

![](SingleAgent.png)


The plot of average rewards for every 100 episodes for **20 Agents** are listed below.

* Episode 37	Average Score: 30.12	Score: 37.74
* Environment solved in 37 episodes!	Average Score: 30.12

![](MultiAgent.png)

## Ideas for Future Work

The Reinforcement Learning agent was trained using Deep Deterministic Policy Gradients

* Further implementation might include improving the accuracy for single agents by varying the frequency and delay of learning as implemented in the NUM_UPDATES and UPDATE_EVERY variables.

* The training time was considerably slow considering that it ran on a GTX 1050 Ti and used barely 1/3rd of the capacity. further optimization of the code might allow faster training and convergence.

* Other algorithms like TRPO, PPO, A3C might have worked in this case as well and might give different or even better performance. Will need to try them out on the same environment and see if they can yield better performance than the current implementation.

* The 20 agents scenario reached the target score much before 100. Might need to run the process for longer periods of time to monitor how much the stability and score is maintained over a longer period of time